# CANCEL TIMES ANALYSES


In [ ]:
%%matlab

subject = 'broca';

projectDate = '2017-01-11';
projectRoot = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_spikes_population';

addpath(genpath(fullfile(projectRoot,'src/code',projectDate)));
dataPath = fullfile(projectRoot,'data',projectDate,subject);



## Population Stop vs Go

In [ ]:

append = false;

opt             = ccm_neuron_stop_vs_go;
opt.append = append;
opt.howProcess  = 'each';
opt.plotFlag    = true;
opt.printPlot    = true;
opt.minTrialPerCond 	= 10;
opt.ms2Std 	= 75;

opt.multiUnit    = multiUnit;

opt.ssrt = ssrtUse;


opt.ANALYZE_CANCELED = true;
opt.ANALYZE_NONCANCELED = false;

opt.parpoolSize = 5;

ccm_neuron_stop_vs_go_pop(subject,projectRoot,projectDate, opt)


## Proportions of cancel times, using mean SDF deflection analysis

In [ ]:
if strcmp(cancelType, 'meanSdf')

    latestTime = 20;  % how late w.r.t. ssrt can a neuron cancel to be considered a "canceling" unit?

    % load a list of neurons sessions and units
    categoryList = {'presacc','presaccNoVis','presaccRamp','visPresacc','presaccPeak'};
    categoryList = {'presacc'};

    % Build a new table of the relevant neurons, and a list of the session/Unit
    for i = 1 : length(categoryList)

        catFileName = fullfile(dataPath, ['ccm_',categoryList{i},'_neurons', addMulti]);
        load(catFileName)

        % load the population of cancel time anlysis
        fileName = fullfile(dataPath, 'go_vs_canceled', ssrtUse, ['ccm_canceled_vs_go_neuronTypes', addMulti]);
        load(fileName)

        % Build a table of the cancelTypes data corresponding to the classic category units
        cancelData = table();
        for j = 1 : size(neurons, 1)
            % find the indices in cancelTypes that correspond to this unit
            iInd = strcmp(neurons.sessionID(j), cancelTypes.sessionID) & strcmp(neurons.unit(j), cancelTypes.unit);
            cancelData = [cancelData; cancelTypes(iInd,:)];
        end


        % Probability of a neuron to have at least one condition that cancels in time
        cancelLogical = cellfun(@(x) x < latestTime, cancelData.cancelTimeSdf, 'uni', false);
        cancelInTimeInd = cell2mat(cellfun(@(x) sum(x), cancelLogical, 'uni', false));
        fprintf('\nProportion of neurons with at least one valid cancel time < %d: %d of %d\t%.2f\n', latestTime, sum(cancelInTimeInd>0), size(neurons, 1), sum(cancelInTimeInd>0)/size(neurons, 1))



        % Which units canceled?
        presaccCancel = neurons(cancelInTimeInd > 0, :);
        presaccNoCancel = neurons(cancelInTimeInd == 0, :);

        load(catFileName)
        neurons = neurons(cancelInTimeInd > 0, :);
        sFileName = fullfile(dataPath, ['ccm_',categoryList{i},'_cancel_',cancelType,'_neurons', addMulti]);
        save(sFileName, 'neurons')


        load(catFileName)
        neurons = neurons(cancelInTimeInd == 0, :);
        sFileName = fullfile(dataPath, ['ccm_',categoryList{i},'_noCancel_',cancelType,'_neurons', addMulti]);
        save(sFileName, 'neurons')

    end
else
    disp('Tyring to do mean SDF analysis but set for some other cancelType')
end


## Proportions of cancel times, using Trial-by-trial SDF deflection analysis

In [ ]:
if strcmp(cancelType, 'trialByTrial')

    latestTime = 20;  % how late w.r.t. ssrt can a neuron cancel to be considered a "canceling" unit?

    % load a list of neurons sessions and units
    categoryList = {'presacc','presaccNoVis','presaccRamp','visPresacc','presaccPeak'};
    categoryList = {'presacc'};

    % Build a new table of the relevant neurons, and a list of the session/Unit
    for i = 1 : length(categoryList)

        catFileName = fullfile(dataPath, ['ccm_',categoryList{i},'_neurons', addMulti]);
        load(catFileName)

        % load the population of cancel time anlysis
        fileName = fullfile(dataPath, 'go_vs_canceled', ssrtUse, ['ccm_canceled_vs_go_neuronTypes', addMulti]);
        load(fileName)

        % Build a table of the cancelTypes data corresponding to the classic category units
        cancelData = table();
        for j = 1 : size(neurons, 1)
            % find the indices in cancelTypes that correspond to this unit
            iInd = strcmp(neurons.sessionID(j), cancelTypes.sessionID) & strcmp(neurons.unit(j), cancelTypes.unit);
            cancelData = [cancelData; cancelTypes(iInd,:)];


        end

        % A neuron "cancels" if one of its condition's mean cancel time is within latestTime
        cancelTimeDistMean = cell(size(cancelData, 1), 1);
        for j = 1 : size(cancelData, 1)
            cancelTimeDistMean{j} = cellfun(@nanmean, cancelData.cancelTimeDist{j});
        end
        cancelData.cancelTimeDistMean = cancelTimeDistMean;

        % Probability of a neuron to have at least one condition that cancels in time
        cancelLogical = cellfun(@(x) x < latestTime, cancelData.cancelTimeDistMean, 'uni', false);
        cancelInTimeInd = cell2mat(cellfun(@(x) sum(x), cancelLogical, 'uni', false));
        fprintf('\nProportion of neurons with at least one valid cancel time < %d: %d of %d\t%.2f\n', latestTime, sum(cancelInTimeInd>0), size(neurons, 1), sum(cancelInTimeInd>0)/size(neurons, 1))



        % Which units canceled?
        presaccCancel = neurons(cancelInTimeInd > 0, :);
        presaccNoCancel = neurons(cancelInTimeInd == 0, :);

        load(catFileName)
        neurons = neurons(cancelInTimeInd > 0, :);
        sFileName = fullfile(dataPath, ['ccm_',categoryList{i},'_cancel_',cancelType,'_neurons', addMulti]);
        save(sFileName, 'neurons')


        load(catFileName)
        neurons = neurons(cancelInTimeInd == 0, :);
        sFileName = fullfile(dataPath, ['ccm_',categoryList{i},'_noCancel_',cancelType,'_neurons', addMulti]);
        save(sFileName, 'neurons')

    end
else
    disp('Tyring to do mean SDF analysis but set for some other cancelType')
end


## Proportions of cancel times, using Hanes&Schall98 criteria

In [ ]:
if strcmp(cancelType, 'meanDifference')

    latestTime = 20;  % how late w.r.t. ssrt can a neuron cancel to be considered a "canceling" unit?

    % load a list of neurons sessions and units
    categoryList = {'presacc','presaccNoVis','presaccRamp','visPresacc','presaccPeak'};
    categoryList = {'presacc'};

    % Build a new table of the relevant neurons, and a list of the session/Unit
    for i = 1 : length(categoryList)

        catFileName = fullfile(dataPath, ['ccm_',categoryList{i},'_neurons', addMulti]);
        load(catFileName)

            % load the population of cancel time anlysis
        fileName = fullfile(dataPath, 'go_vs_canceled', ssrtUse, ['ccm_canceled_vs_go_neuronTypes', addMulti]);
        load(fileName)


        cancelData = table();
        for j = 1 : size(neurons, 1)
            % find the indices in cancelTypes that correspond to this unit
            iInd = strcmp(neurons.sessionID(j), cancelTypes.sessionID) & strcmp(neurons.unit(j), cancelTypes.unit);
            cancelData = [cancelData; cancelTypes(iInd,:)];


        end




        % How many (among all units X SSDs were go vs stop different during 40 ms peri-SSRT?
        alphaVal = .05;
        peri40msInd = cellfun(@(x) x < alphaVal, cancelData.pValue40msStopStop, 'uni', false);
        notPeri40ms = cellfun(@(x) x >= alphaVal, cancelData.pValue40msStopStop, 'uni', false);

        pPeri40ms = sum(cell2mat(peri40msInd)) / (sum(cell2mat(peri40msInd)) + sum(cell2mat(notPeri40ms)));





        %   START HERE TO USE cancelTime2Std TO SORT NEURONS
        cancelData.cancelTime = cellfun(@(x,y,z) x - y - z, cancelData.cancelTime2Std, cancelData.stopStopSsd, cancelData.stopStopSsrt, 'uni', false);


        % How many conditions out of all possible had a cancel time?
        allCancelTime = cell2mat(cellfun(@(x) ~isnan(x), cancelData.cancelTime, 'uni', false));
        nCancelCond = sum(allCancelTime);
        allCancelPreSsrt = cell2mat(cellfun(@(x) x < latestTime, cancelData.cancelTime, 'uni', false));


        nCondTotal = size(cell2mat(cancelData.cancelTime), 1);
        pCancelCond = nCancelCond / nCondTotal;
        fprintf('\nProportion of valid cancel times per valid conditions: %d of %d:\t%.3f\n', nCancelCond, nCondTotal, pCancelCond)


        pCancelPreSsrt = sum(allCancelPreSsrt) / nCancelCond;
        fprintf('\nProportion of cancel times < %d ms per valid conditions:: %d of %d\t%.3f\n', latestTime, sum(allCancelPreSsrt), nCondTotal, pCancelPreSsrt)


        % What's the full distribution of cancel times?
        allCancelTime = cell2mat(cancelData.cancelTime);
        cancelCond = allCancelTime(~isnan(allCancelTime));



        % How many conditions canceled for this neuron?
        nCancelPerNeuron = cellfun(@(x) sum(~isnan(x)), cancelData.cancelTime);
        % How many potential conditions to cancel for this neuron?
        nCondPerNeuron = cellfun(@length, cancelData.cancelTime);

        % If any condition canceled for this neuron, count it as a "cancel" neuron
        neuronsCanceled = nCancelPerNeuron > 0;

        % Probability of a neuron to have at least one condition that cancels
        pNeuronCond = sum(neuronsCanceled) / size(neurons, 1);
        fprintf('\nProportion of neurons with at least one valid cancel time: %d of %d\t%.2f\n', sum(neuronsCanceled), size(neurons, 1), pNeuronCond)


        % Probability of a neuron to have at least one condition that cancels in time
        cancelLogical = cellfun(@(x) x < latestTime, cancelData.cancelTime, 'uni', false);
        cancelInTimeInd = cell2mat(cellfun(@(x) sum(x), cancelLogical, 'uni', false));
        fprintf('\nProportion of neurons with at least one valid cancel time < %d: %d of %d\t%.2f\n', latestTime, sum(cancelInTimeInd>0), size(neurons, 1), sum(cancelInTimeInd>0)/size(neurons, 1))




        % Which units canceled?
        presaccCancel = neurons(cancelInTimeInd > 0, :);
        presaccNoCancel = neurons(cancelInTimeInd == 0, :);

        load(catFileName)
        neurons = neurons(cancelInTimeInd > 0, :);
        sFileName = fullfile(dataPath, ['ccm_',categoryList{i},'_cancel_',cancelType,'_neurons', addMulti]);
        save(sFileName, 'neurons')


        load(catFileName)
        neurons = neurons(cancelInTimeInd == 0, :);
        sFileName = fullfile(dataPath, ['ccm_',categoryList{i},'_noCancel_',cancelType,'_neurons', addMulti]);
        save(sFileName, 'neurons')

    end
else
    disp('Tyring to do mean SDF analysis but set for some other cancelType')
end


# Neuronal Separation Distributions

In [ ]:
opt             = ccm_options;
opt.plotFlag    = true;
opt.printPlot    = true;

opt.multiUnit    = multiUnit;

opt.ssrt = ssrtUse;

opt.neuronCategory = {'presaccRamp'};


data = ccm_cancel_time_analyses_pop(subject, projectRoot, projectDate, opt); 





### Write data to a csv file

In [ ]:
cancelData = data.cancelData;

cancelTime = cell2mat(cellfun(@(x,y,z) x - y - z, cancelData.cancelTime2Std, cancelData.stopStopSsd, cancelData.stopStopSsrt, 'uni', false));

sessionID = [];
unit = [];
rf = [];
for i = 1 : size(cancelData, 1)
    nRep = size(cancelData.stopStopSsd{i}, 1);
    sessionID = [sessionID; repmat(cancelData.sessionID(i), nRep, 1)];
    unit = [unit; repmat(cancelData.unit(i), nRep, 1)];
    rf = [rf; repmat(cancelData.rf(i), nRep, 1)];
end
ssd = cell2mat(cancelData.stopStopSsd);
ssrt = cell2mat(cancelData.stopStopSsrt);
coherence = cell2mat(cancelData.stopStopCoh);
easyHard = cell2mat(cancelData.stopStopCond);

data = table(sessionID, unit, rf, ssd, ssrt, coherence, easyHard, cancelTime);

fileName =  fullfile(dataPath,['cancelData_'cancelType,'_',opt.neuronCategory{1}, addMulti,'.csv'])
writetable(data,fileName)


# Neuronal Separation Per Session

In [ ]:
neuronCategory = {'presacc'};

fileName = fullfile(dataPath, ['ccm_',neuronCategory{1},'_neurons', addMulti]);
load(fileName)

opt             = ccm_neuron_stop_vs_go;
opt.ms2Std  = 75;
opt.howProcess  = 'each';
opt.plotFlag    = true;
opt.printPlot    = true;
opt.minTrialPerCond 	= 10;

opt.multiUnit 	= multiUnit;

opt.ssrt 	= ssrtUse;

poolID = parpool(4);
parfor i = 1 : size(neurons, 1)
    iData = ccm_neuron_stop_vs_go(subject, neurons.sessionID{i}, neurons.unit(i), opt);
    % fprintf('total process time: %.2f\n', toc)
end
delete(poolID)


#PROCEED TO CHOICE_CANCEL ANALYSES NOTEBOOK